In [17]:
%pip install mediapipe opencv-python requests

Note: you may need to restart the kernel to use updated packages.


# Pre processing training data 
- Go through all lines in training.txt
- Get class and url location 
- If class not in 5 I want -> Continue
- Go to url file location
    - Try to download file 
    - If fail contrinue 
    - Store downloaded img in tmp location 
    - Perform processing on image 
    - Delete image from tmp location 
- Calculate angles once landmarks from image are proccesd
- Store angles of landmarks in csv 

In [45]:
a = {1:2}
a |= {2:1} 
a

{1: 2, 2: 1}

In [73]:
import cv2
import mediapipe as mp
import numpy as np
import os
import urllib 
import requests
import csv
from PIL import Image 
from PIL import ImageFile

import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

min_detection_confidence = 0.5
min_tracking_confidence = 0.5

# All landmark except for hand and face specific
RelevantLandmarks = list(mp_pose.PoseLandmark)[11:17] + list(mp_pose.PoseLandmark)[23:]

path = 'C:\\Users\\moham\\Desktop\\Third Year Project\\yoga82code\\Yoga-82\\'
img_links_path = os.path.join(path, 'yoga_dataset_links')
train_file = os.path.join(path, 'yoga_train.txt')
test_file = os.path.join(path, 'yoga_test.txt')


class_map = {
    (0,2,75) : "Warrior", #Warrior
    (0,0,68) : "Tree", #Tree 
    (4,14,68): "Cobra", #Cobra
    (4,16,44): "Plank", #Plank 
    (0,1,17): "DownDog" #Downward Dog
}


In [74]:
def generate_link_map(train=True):
    train_links = dict()
    with open(train_file if train else test_file, 'r') as file:
        for line in file:
            img_path, l1, l2, l3 = line.split(',')

            #Ending early if img not classified as what we want 
            img_class = (int(l1), int(l2), int(l3))

            if img_class not in class_map:
                continue 

            img_path, img_num = img_path.replace('/', ' ').split(' ') 
            img_path = img_path + '.txt'
            
            img_to_find = img_path.replace('.txt', '/'+img_num)
            
            #Add more links to dictionary 
            train_links |= {inner_line.split('\t')[0] : inner_line.split('\t')[1].strip() 
                            for inner_line in open(os.path.join(img_links_path, img_path))}
    return train_links 
training_img_map = generate_link_map(train=True)
testing_img_map = generate_link_map(train=False)

In [89]:
RelevantLandmarks

[<PoseLandmark.LEFT_SHOULDER: 11>,
 <PoseLandmark.RIGHT_SHOULDER: 12>,
 <PoseLandmark.LEFT_ELBOW: 13>,
 <PoseLandmark.RIGHT_ELBOW: 14>,
 <PoseLandmark.LEFT_WRIST: 15>,
 <PoseLandmark.RIGHT_WRIST: 16>,
 <PoseLandmark.LEFT_HIP: 23>,
 <PoseLandmark.RIGHT_HIP: 24>,
 <PoseLandmark.LEFT_KNEE: 25>,
 <PoseLandmark.RIGHT_KNEE: 26>,
 <PoseLandmark.LEFT_ANKLE: 27>,
 <PoseLandmark.RIGHT_ANKLE: 28>,
 <PoseLandmark.LEFT_HEEL: 29>,
 <PoseLandmark.RIGHT_HEEL: 30>,
 <PoseLandmark.LEFT_FOOT_INDEX: 31>,
 <PoseLandmark.RIGHT_FOOT_INDEX: 32>]

In [97]:
def generate_csv_train(train=True):
    with open(train_file if train else test_file, 'r') as file:
        lines = file.readlines()
    
    img_map = training_img_map if train else testing_img_map 
    
    with open(os.path.join(path, "training.csv" if train else "testing.csv"), 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

        for line in lines:
            img_found_flag = False 
            
            img_path, l1, l2, l3 = line.split(',')
            
            #Ending early if img not classified as what we want 
            img_class = (int(l1), int(l2), int(l3))
            
            if img_class not in class_map:
                continue
            
            # Ending early if for some reason I havn't included this image 
            # In our previous search 
            if img_path not in img_map:
                print("IMAGE NOT IN MAP.. ERROR?")
                return 
                
            img_url = img_map[img_path] 
            tmp_img = os.path.join(path,"tmp.jpg")
            
            try:
                img_data = requests.get(img_url).content
            except Exception as e:
                print(f'Error in downloading image... {img_url} | Trying next image...\n')
                print(e)
                continue 
            
            with open(tmp_img, 'wb') as handler:
                handler.write(img_data)
            
            try:
                image = cv2.imread(tmp_img)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                os.remove(tmp_img)
            except Exception as e:
                print("COULDN'T CONVERT IMAGE TO CV2 ARRAY.. Trying next img\n")
                continue
            except FileNodeFoundError as e:
                print("Couldn't find temp file.. Trying next img\n")
                continue 
            
            
            print("Succesfully download and read image from URL")       

            # Initialize fresh pose tracker and run it.
            with mp_pose.Pose(min_detection_confidence=min_detection_confidence, min_tracking_confidence=min_tracking_confidence) as pose_tracker:
                result = pose_tracker.process(image=image)
                pose_landmarks = result.pose_landmarks
                


            #If a one of the valid pose' was detected, write this  
            output_image = image.copy()

            if pose_landmarks is not None:
                print("Succesfully generated pose landmarks from url image")
                
                #Getting pose landmark information (Only for relevant landmarks, to reduce our dimensions)
                pose_relevant_landmark_cords = [[pose_landmarks.landmark[kp.value].x, pose_landmarks.landmark[kp.value].y]
                                               for kp in RelevantLandmarks]
                
                print(f"Example Landmark Cords {pose_relevant_landmark_cords[0]}")
                
                # Map pose landmarks from [0, 1] range to absolute coordinates to get
                # correct aspect ratio.
                frame_height, frame_width = output_image.shape[:2]
                print(f"Succesfully extracted height and width of frame: {(frame_height, frame_width)}")
                pose_relevant_landmark_cords *= np.array([frame_width, frame_height])

                # Write pose sample to CSV.
                pose_relevant_landmark_cords = np.around(pose_relevant_landmark_cords, 5).flatten().astype(str).tolist()
                csv_out_writer.writerow([class_map[img_class]] + pose_relevant_landmark_cords)
                print("!!! Successfully added example row to CSV !!!\n")
                
                
                '''
                    Instead of storing X,Y Cord of each landmark 
                    I should instead store angle of each relevant keypoint 
                
                '''
                
            else:
                print(f"Could not extract pose from image: {img_url} Trying next image...\n")
generate_csv_train()

Succesfully download and read image from URL
Could not extract pose from image: https://thumb7.shutterstock.com/display_pic_with_logo/2907205/267890807/stock-photo-yoga-pose-adho-mukha-svanasana-downward-facing-dog-poseon-watercolor-style-background-267890807.jpg Trying next image...

Succesfully download and read image from URL
Succesfully generated pose landmarks from url image
Example Landmark Cords [0.623866081237793, 0.5118217468261719]
16
Succesfully extracted height and width of frame: (470, 300)
!!! Successfully added example row to CSV !!!

Succesfully download and read image from URL
Could not extract pose from image: http://i.ytimg.com/vi/stv0C6qT-Vc/hqdefault.jpg Trying next image...

Succesfully download and read image from URL
Succesfully generated pose landmarks from url image
Example Landmark Cords [0.44685041904449463, 0.578008770942688]
16
Succesfully extracted height and width of frame: (600, 800)
!!! Successfully added example row to CSV !!!

COULDN'T CONVERT IMAGE 

Error in downloading image... http://www.thenerdyyogi.com/wp-content/uploads/2015/06/Down-Dog-in-Stowe-VT-BLOG-1.png | Trying next image...

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error in downloading image... http://aboutyoga.in/wp-content/uploads/2013/12/adhomukhasvanasana1.jpg | Trying next image...

Exceeded 30 redirects.
Succesfully download and read image from URL
Succesfully generated pose landmarks from url image
Example Landmark Cords [0.55743807554245, 0.37598592042922974]
16
Succesfully extracted height and width of frame: (320, 450)
!!! Successfully added example row to CSV !!!

Succesfully download and read image from URL
Could not extract pose from image: https://i.pinimg.com/736x/49/b0/f5/49b0f52bf8870c151230c9008dcf799c--inner-peace-dogs.jpg Trying next image...

Error in downloading image... http://2cxvx317i0xp2p56kk3rqg5w-wpengine.netdna-ssl.com/wp-content/uploads/2010/10/shanna-downward-facing-dog.jpg | Trying nex

KeyboardInterrupt: 

In [30]:
def calc_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)    
    c = np.array(c)   
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 380-angle 
    
    return angle 